In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


import pandas as pd


players_df = pd.read_csv('/content/drive/My Drive/Dissertation_Datasets/male_players.csv')
teams_df = pd.read_csv('/content/drive/My Drive/Dissertation_Datasets/male_teams.csv')

# Verify loading by displaying the first few rows of each dataset
print("Male Players:")
print(players_df.head())

print("\n Male Teams:")
print(teams_df.head())

Mounted at /content/drive


<ipython-input-1-ec4838e48911>:9: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_df = pd.read_csv('/content/drive/My Drive/Dissertation_Datasets/male_players.csv')


Male Players:
   player_id                             player_url  fifa_version  \
0     231747    /player/231747/kylian-mbappe/240002          24.0   
1     239085   /player/239085/erling-haaland/240002          24.0   
2     192985  /player/192985/kevin-de-bruyne/240002          24.0   
3     158023     /player/158023/lionel-messi/240002          24.0   
4     165153    /player/165153/karim-benzema/240002          24.0   

   fifa_update update_as_of    short_name                       long_name  \
0          2.0   2023-09-22     K. Mbappé            Kylian Mbappé Lottin   
1          2.0   2023-09-22    E. Haaland            Erling Braut Haaland   
2          2.0   2023-09-22  K. De Bruyne                 Kevin De Bruyne   
3          2.0   2023-09-22      L. Messi  Lionel Andrés Messi Cuccittini   
4          2.0   2023-09-22    K. Benzema                   Karim Benzema   

  player_positions  overall  potential  ...   ldm   cdm   rdm   rwb    lb  \
0           ST, LW       91    

In [ ]:
import pandas as pd
import numpy as np


# Get all unique nationalities and their coordinates
unique_nationalities = players_df['nationality_name'].dropna().unique()
print(unique_nationalities)

coordinates = {
    "Argentina": {"latitude": -34.603722, "longitude": -58.381592},
    "Australia": {"latitude": -25.274398, "longitude": 133.775136},
    "Austria": {"latitude": 47.516231, "longitude": 14.550072},
    "Belgium": {"latitude": 50.850340, "longitude": 4.351710},
    "Brazil": {"latitude": -14.235004, "longitude": -51.925280},
    "Canada": {"latitude": 56.130366, "longitude": -106.346771},
    "Chile": {"latitude": -35.675147, "longitude": -71.542969},
    "China": {"latitude": 35.861660, "longitude": 104.195397},
    "Colombia": {"latitude": 4.570868, "longitude": -74.297333},
    "Croatia": {"latitude": 45.1, "longitude": 15.2},
    "Czech Republic": {"latitude": 49.817492, "longitude": 15.472962},
    "Denmark": {"latitude": 56.263920, "longitude": 9.501785},
    "England": {"latitude": 51.509865, "longitude": -0.118092},
    "Finland": {"latitude": 61.924110, "longitude": 25.748151},
    "France": {"latitude": 48.864716, "longitude": 2.349014},
    "Germany": {"latitude": 52.520008, "longitude": 13.404954},
    "Greece": {"latitude": 39.074208, "longitude": 21.824312},
    "Hungary": {"latitude": 47.162494, "longitude": 19.503304},
    "Iceland": {"latitude": 64.963051, "longitude": -19.020835},
    "India": {"latitude": 20.593684, "longitude": 78.962880},
    "Ireland": {"latitude": 53.412910, "longitude": -8.243890},
    "Italy": {"latitude": 41.871941, "longitude": 12.567380},
    "Japan": {"latitude": 36.204824, "longitude": 138.252924},
    "Mexico": {"latitude": 23.634501, "longitude": -102.552784},
    "Netherlands": {"latitude": 52.132633, "longitude": 5.291266},
    "New Zealand": {"latitude": -40.900557, "longitude": 174.885971},
    "Norway": {"latitude": 60.472024, "longitude": 8.468946},
    "Poland": {"latitude": 51.919438, "longitude": 19.145136},
    "Portugal": {"latitude": 39.399872, "longitude": -8.224454},
    "Russia": {"latitude": 61.524010, "longitude": 105.318756},
    "Scotland": {"latitude": 56.490671, "longitude": -4.202646},
    "South Africa": {"latitude": -30.559482, "longitude": 22.937506},
    "South Korea": {"latitude": 35.907757, "longitude": 127.766922},
    "Spain": {"latitude": 40.463667, "longitude": -3.749220},
    "Sweden": {"latitude": 60.128161, "longitude": 18.643501},
    "Switzerland": {"latitude": 46.818188, "longitude": 8.227512},
    "Turkey": {"latitude": 38.963745, "longitude": 35.243322},
    "Ukraine": {"latitude": 48.379433, "longitude": 31.165580},
    "United States": {"latitude": 37.090240, "longitude": -95.712891},
    "Uruguay": {"latitude": -32.522779, "longitude": -55.765835},
    "Wales": {"latitude": 52.130661, "longitude": -3.783712},
    "Zimbabwe": {"latitude": -19.015438, "longitude": 29.154857},
}

# Map latitude and longitude to the players dataset
players_df['latitude'] = players_df['nationality_name'].map(lambda x: coordinates.get(x, {}).get('latitude', 0))
players_df['longitude'] = players_df['nationality_name'].map(lambda x: coordinates.get(x, {}).get('longitude', 0))

# Extracting Bristol City Players
bristol_fc_players = players_df[players_df['club_name'] == "Bristol City"].copy()

# Ensure we have only the latest version and update for each player
latest_version = players_df['fifa_version'].max()
latest_update = players_df[players_df['fifa_version'] == latest_version]['fifa_update'].max()
bristol_fc_players = bristol_fc_players[
    (bristol_fc_players['fifa_version'] == latest_version) &
    (bristol_fc_players['fifa_update'] == latest_update)
]
# Map positions to pitch roles for a 3-4-3 formation
formation_positions = {
    "GK": 1,       # 1 Goalkeeper
    "DEF": 3,      # 3 Defenders
    "MID": 4,      # 4 Midfielders
    "FWD": 3       # 3 Forwards
}

# Helper function to map multiple positions to general categories
def map_to_general_position(player_positions):
    position_mapping = {
        "GK": "GK",
        "ST": "FWD", "LW": "FWD", "RW": "FWD", "CF": "FWD",
        "CM": "MID", "CDM": "MID", "CAM": "MID", "LM": "MID", "RM": "MID",
        "CB": "DEF", "LB": "DEF", "RB": "DEF", "LWB": "DEF", "RWB": "DEF"
    }
    primary_position = player_positions.split(',')[0].strip()
    return position_mapping.get(primary_position, "OTHER")

# Add general position column
bristol_fc_players['general_position'] = bristol_fc_players['player_positions'].map(map_to_general_position)

# Sort Bristol City players by overall rating in descending order
bristol_fc_players = bristol_fc_players.sort_values(by='overall', ascending=False)

# Function to assign players to starting 11 and bench
def assign_players(players, formation):
    starting_11 = []
    bench = []
    remaining_players = players.copy()

    # Loop through formation requirements
    for position, count in formation.items():
        # Filter players for the current position
        position_players = remaining_players[remaining_players['general_position'] == position]

        # Select the top players for the position
        selected = position_players.head(count)
        starting_11.extend(selected.index.tolist())

        # Remove selected players from remaining players
        remaining_players = remaining_players.drop(selected.index)

    # Assign the remaining players to the bench
    bench.extend(remaining_players.index.tolist())
    return players.loc[starting_11], players.loc[bench]

# Assign starting 11 and bench players for Bristol City
starting_11, bench = assign_players(bristol_fc_players, formation_positions)

# Save results to CSV files
starting_11.to_csv('bristol_city_starting_11.csv', index=False)
bench.to_csv('bristol_city_bench.csv', index=False)

# Download files
from google.colab import files
files.download('bristol_city_starting_11.csv')
files.download('bristol_city_bench.csv')





['France' 'Norway' 'Belgium' 'Argentina' 'Poland' 'England' 'Brazil'
 'Spain' 'Portugal' 'Germany' 'Netherlands' 'Egypt' 'Uruguay' 'Nigeria'
 'Slovenia' 'Italy' 'Switzerland' 'Croatia' 'Korea Republic' 'Georgia'
 'Algeria' 'Senegal' 'Scotland' 'Serbia' 'Cameroon' 'Costa Rica' 'Austria'
 'Turkey' 'Morocco' 'Ghana' 'Denmark' 'Colombia' 'Hungary' 'Slovakia'
 'Canada' 'Czech Republic' 'Bosnia and Herzegovina' "Côte d'Ivoire"
 'Ukraine' 'Montenegro' 'Iran' 'Sweden' 'Burkina Faso' 'Mexico' 'Greece'
 'Tunisia' 'Japan' 'Libya' 'Chile' 'Finland' 'Armenia'
 'Central African Republic' 'Paraguay' 'Mozambique' 'Kosovo'
 'North Macedonia' 'Ecuador' 'Congo DR' 'Gabon' 'Mali' 'United States'
 'Jamaica' 'Russia' 'Malta' 'Peru' 'Angola' 'Australia' 'Guinea'
 'Suriname' 'Togo' 'Albania' 'Gambia' 'Israel' 'Trinidad and Tobago'
 'Zimbabwe' 'Wales' 'Venezuela' 'Saudi Arabia' 'Romania'
 'Republic of Ireland' 'Luxembourg' 'Liberia' 'Bulgaria' 'China PR'
 'Iceland' 'Qatar' 'Equatorial Guinea' 'Burundi' 'Zambia

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>